**NLP**

In [1]:
import os 
from dotenv import load_dotenv
#we can use override = True inside parenthesis to override any env variable set systemwide
load_dotenv()

True

In [2]:
nlp_key = os.getenv("nlp_key")
user = os.getenv("user_name")

In [3]:
print(user)

Chandan Sengupta


In [17]:
#openai test
from openai import OpenAI
client=OpenAI(api_key = nlp_key)

In [19]:
# response = client.chat.completions.create(
#     model="gpt-3.5-turbo",
#     messages=[
#         {"role":"user",
#          "content":"write two lines about leo messi"}
#     ]
# )

In [2]:
# # Step 1: Install Necessary Libraries
# # Run this cell only if the libraries are not already installed
# !pip install nltk transformers torch sentencepiece

# Step 2: Import Libraries and Download NLTK Data
import nltk
from nltk import pos_tag, word_tokenize
from nltk.chunk import ne_chunk
from transformers import MarianMTModel, MarianTokenizer
import warnings
warnings.filterwarnings('ignore')

# Download necessary NLTK data files
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

# Step 3: Define the Function to Extract Entities
def extract_entities(sentence):
    tokens = word_tokenize(sentence)
    pos_tags = pos_tag(tokens)
    named_entities = ne_chunk(pos_tags, binary=True)
    
    entities = []
    for chunk in named_entities:
        if hasattr(chunk, 'label') and chunk.label() == 'NE':
            entity = ' '.join(c[0] for c in chunk)
            entities.append(entity)
    
    return entities, pos_tags

# Step 4: Define the Function to Translate with Preserved Entities
def translate_with_preserved_entities(sentence, src_lang='en', tgt_lang='fr'):
    # Extract entities and POS tags
    entities, pos_tags = extract_entities(sentence)
    
    # Replace entities with placeholders
    modified_sentence = sentence
    entity_map = {}
    for i, entity in enumerate(entities):
        placeholder = f"ENTITY_{i}"
        entity_map[placeholder] = entity
        modified_sentence = modified_sentence.replace(entity, placeholder)
    
    # Load translation model and tokenizer
    model_name = f'Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}'
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    
    # Translate the modified sentence
    tokenized_text = tokenizer(modified_sentence, return_tensors='pt')
    translated_tokens = model.generate(**tokenized_text)
    translated_sentence = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    
    # Replace placeholders with original entities
    for placeholder, entity in entity_map.items():
        translated_sentence = translated_sentence.replace(placeholder, entity)
    
    return translated_sentence

# Step 5: Translate a Given Sentence
sentence = "Chandan went to cognizant office on Sunday at 11.58 pm"
translated_sentence = translate_with_preserved_entities(sentence)
print(f"Original Sentence: {sentence}")
print(f"Translated Sentence: {translated_sentence}")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sengu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sengu\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\sengu\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\sengu\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


Original Sentence: Chandan went to cognizant office on Sunday at 11.58 pm
Translated Sentence: Chandan est allé au bureau de connaissance le dimanche à 23h58
